In [1]:
# moviepy editor example
from moviepy.editor import 
    VideoFileClip
    concatenate_videoclips
    vfx # video fx
    afx # audio fx
    CompositeAudioClip # 

clip1 = VideoFileClip("").subclip(0, 10)
clip2 = VideoFileClip("").subclip(10, 20)
clip3 = VideoFileClip("").subclip(20, 30)
clip4 = VideoFileClip("").subclip(20, 30).fx(vfx.colorx, 1.5)\
    .fx(vfx.lum_contrast, 0, 50, 128)

combined = concatenate_videoclips([clip1, clip2, clip3, ,clip4])
combined.write_videogile("sequence-clips.mp4")

VideoFileClip().fx(vfx.fadein, 1).fx(vfx.fadeout, 1)


SyntaxError: invalid syntax (<ipython-input-1-0585f82afcb7>, line 1)

# NDI-PY

# NDI Stream Capture in Jupyter Notebook

An improved version of your NDI streaming notebook with better error handling, cleaner threading, and more robust frame updates.

In [ ]:
## 📦 Cell 1: Install Dependencies

# Install required packages
!pip install ndi-python opencv-python ipywidgets
# Note: Ensure NDI SDK/Runtime is installed on your system

In [ ]:
## 🧠 Cell 2: Imports & Setup

import threading
import time
import cv2
import numpy as np
import ndi
import ipywidgets as widgets
from IPython.display import display, clear_output
import queue
from contextlib import contextmanager

# Global state management
class NDIStreamState:
    def __init__(self):
        self.frame_queue = queue.Queue(maxsize=5)  # Prevent memory buildup
        self.stop_event = threading.Event()
        self.receiver_thread = None
        self.display_thread = None
        
    def stop_all(self):
        self.stop_event.set()
        if self.receiver_thread and self.receiver_thread.is_alive():
            self.receiver_thread.join(timeout=2)
        if self.display_thread and self.display_thread.is_alive():
            self.display_thread.join(timeout=2)

stream_state = NDIStreamState()

In [ ]:
## 🔄 Cell 3: Enhanced NDI Receiver

def ndi_receiver_loop(state):
    """Background thread for NDI frame capture with proper error handling"""
    try:
        # Initialize NDI
        if not ndi.initialize():
            print("Failed to initialize NDI")
            return
            
        print("NDI initialized successfully")
        
        # Find NDI sources
        sources = ndi.find_sources(timeout=5000)
        if not sources:
            print("No NDI sources found")
            return
            
        print(f"Found {len(sources)} NDI sources:")
        for i, source in enumerate(sources):
            print(f"  {i}: {source.name}")
        
        # Create receiver
        receiver = ndi.Receiver()
        receiver.connect(sources[0])  # Connect to first source
        print(f"Connected to: {sources[0].name}")
        
        frame_count = 0
        while not state.stop_event.is_set():
            try:
                # Capture frame with timeout
                frame = receiver.read(timeout=1000)  # 1 second timeout
                
                if frame is not None:
                    frame_count += 1
                    # Convert to RGB if needed
                    if frame.shape[2] == 4:  # RGBA
                        frame = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)
                    elif frame.shape[2] == 3 and frame.dtype == np.uint8:
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    
                    # Add frame to queue (non-blocking)
                    try:
                        state.frame_queue.put_nowait({
                            'frame': frame,
                            'timestamp': time.time(),
                            'frame_number': frame_count
                        })
                    except queue.Full:
                        # Remove old frame and add new one
                        try:
                            state.frame_queue.get_nowait()
                            state.frame_queue.put_nowait({
                                'frame': frame,
                                'timestamp': time.time(),
                                'frame_number': frame_count
                            })
                        except queue.Empty:
                            pass
                            
            except Exception as e:
                print(f"Frame capture error: {e}")
                time.sleep(0.1)
                
    except Exception as e:
        print(f"NDI receiver error: {e}")
    finally:
        try:
            ndi.destroy()
        except:
            pass
        print("NDI receiver stopped")

In [ ]:
## 🖼️ Cell 4: Display Widget with Controls

# Create widgets
image_widget = widgets.Image(
    format='jpeg',  # Better compression than raw
    width=640,
    height=360
)

# Info display
info_label = widgets.HTML(value="<b>Status:</b> Initializing...")

# Control buttons
start_button = widgets.Button(description="Start Stream", button_style='success')
stop_button = widgets.Button(description="Stop Stream", button_style='danger')
snapshot_button = widgets.Button(description="Save Snapshot", button_style='info')

# Layout
controls = widgets.HBox([start_button, stop_button, snapshot_button])
display_box = widgets.VBox([
    info_label,
    image_widget,
    controls
])

display(display_box)

# Global variables for tracking
current_frame = None
frame_stats = {'count': 0, 'fps': 0, 'last_time': time.time()}

In [ ]:
## 🚀 Cell 5: Display Update Loop

def display_update_loop(state):
    """Update display widget with latest frames"""
    global current_frame, frame_stats
    
    while not state.stop_event.is_set():
        try:
            # Get latest frame (non-blocking)
            frame_data = state.frame_queue.get(timeout=0.1)
            current_frame = frame_data['frame']
            
            # Resize for display
            display_frame = cv2.resize(current_frame, (640, 360))
            
            # Convert to JPEG bytes for widget
            _, buffer = cv2.imencode('.jpg', display_frame, [cv2.IMWRITE_JPEG_QUALITY, 80])
            image_widget.value = buffer.tobytes()
            
            # Update stats
            frame_stats['count'] += 1
            current_time = time.time()
            time_diff = current_time - frame_stats['last_time']
            if time_diff >= 1.0:  # Update FPS every second
                frame_stats['fps'] = frame_stats['count'] / time_diff
                frame_stats['count'] = 0
                frame_stats['last_time'] = current_time
            
            # Update info
            info_label.value = f"""
            <b>Status:</b> Streaming | 
            <b>FPS:</b> {frame_stats['fps']:.1f} | 
            <b>Frame:</b> {frame_data['frame_number']} |
            <b>Resolution:</b> {current_frame.shape[1]}x{current_frame.shape[0]}
            """
            
        except queue.Empty:
            continue
        except Exception as e:
            print(f"Display update error: {e}")
            time.sleep(0.1)

In [ ]:
## 🎮 Cell 6: Button Event Handlers

def start_streaming(_):
    """Start NDI streaming"""
    global stream_state
    
    if stream_state.receiver_thread and stream_state.receiver_thread.is_alive():
        print("Stream already running!")
        return
    
    # Reset state
    stream_state = NDIStreamState()
    
    # Start threads
    stream_state.receiver_thread = threading.Thread(
        target=ndi_receiver_loop, 
        args=(stream_state,), 
        daemon=True
    )
    stream_state.display_thread = threading.Thread(
        target=display_update_loop, 
        args=(stream_state,), 
        daemon=True
    )
    
    stream_state.receiver_thread.start()
    stream_state.display_thread.start()
    
    info_label.value = "<b>Status:</b> Starting stream..."
    print("NDI stream started")

def stop_streaming(_):
    """Stop NDI streaming"""
    global stream_state
    stream_state.stop_all()
    info_label.value = "<b>Status:</b> Stopped"
    print("NDI stream stopped")

def save_snapshot(_):
    """Save current frame as image"""
    global current_frame
    if current_frame is not None:
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        filename = f"ndi_snapshot_{timestamp}.jpg"
        cv2.imwrite(filename, cv2.cvtColor(current_frame, cv2.COLOR_RGB2BGR))
        print(f"Snapshot saved: {filename}")
    else:
        print("No frame available to save")

# Connect button events
start_button.on_click(start_streaming)
stop_button.on_click(stop_streaming)
snapshot_button.on_click(save_snapshot)

In [ ]:
## 🛑 Cell 7: Cleanup (Run when done)

# Cleanup function
def cleanup_ndi():
    global stream_state
    stream_state.stop_all()
    # Clear display
    image_widget.value = b''
    info_label.value = "<b>Status:</b> Cleaned up"
    print("NDI streaming cleaned up")

# Auto-cleanup when notebook kernel is interrupted
import atexit
atexit.register(cleanup_ndi)

# Manual cleanup
# cleanup_ndi()  # Uncomment to run cleanup

## 💡 Key Improvements

1. **Better Threading**: Separate threads for capture and display with proper synchronization
1. **Queue Management**: Prevents memory buildup with frame queue limiting
1. **Error Handling**: Comprehensive try-catch blocks and timeout handling
1. **User Controls**: Start/stop buttons and snapshot functionality
1. **Performance Stats**: Real-time FPS and frame counter display
1. **Proper Cleanup**: Ensures threads are stopped cleanly

## 🔧 Troubleshooting Tips

- **No NDI Sources**: Ensure NDI-enabled devices are on the same network
- **Poor Performance**: Reduce frame rate or resolution in the NDI source
- **Memory Issues**: The queue system prevents buildup, but monitor usage
- **Threading Issues**: Always use the stop button rather than kernel restart

## 🚀 Next Steps

- Add frame processing filters (blur, edge detection, etc.)
- Implement recording functionality
- Add multiple source selection
- Create custom overlays or annotations